In [1]:
import pytorch_lightning as pl
from  data.data_loader import MyDataModule
from models import OneLayer
from losses.CrossEntropy import CrossEntropy
from training.PL_train import Main_Loop
from metrics.accuracy import accuracy
CUDA_LAUNCH_BLOCKING=1

In [2]:
model = OneLayer(n_classes=5).cuda()
criterion = CrossEntropy().cuda()
data_module = MyDataModule()

In [3]:
trainer = pl.Trainer(gpus=1,max_epochs = 10)
main = Main_Loop(model = model,loss= criterion, metric= accuracy)
trainer.fit(main, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | OneLayer     | 10    
1 | loss  | CrossEntropy | 0     
---------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]